## Regressão Linear com Databricks e MLlib: Análise de Dados Automotivos

### Introdução
Neste notebook, realizaremos uma análise de regressão linear utilizando o conjunto de dados de automóveis (Carros.csv). Nosso objetivo é prever a potência do motor (HP) com base em outras características dos veículos, como consumo de combustível (Consumo), número de cilindros (Cilindros) e cilindrada do motor (Cilindradas). Utilizaremos a biblioteca MLlib, nativa do Apache Spark, para construir e avaliar o modelo de regressão linear.

A regressão linear é uma técnica estatística utilizada para modelar a relação entre uma variável dependente e uma ou mais variáveis independentes. No contexto deste notebook, HP será nossa variável dependente, e as demais características dos veículos serão as variáveis independentes.

### Desenvolvimento
#### Importação das Bibliotecas e Carregamento dos Dados
Primeiro, importamos as bibliotecas necessárias para a execução das tarefas de Machine Learning, incluindo as classes para regressão linear, avaliação de regressão e montagem de vetores de características.

In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

Em seguida, carregamos o conjunto de dados Carros.csv em um DataFrame do Spark, permitindo a manipulação e análise dos dados.

In [0]:
#importa
Carros_temp = spark.read.csv("/FileStore/tables/Carros.csv",inferSchema=True, header=True, sep=";")
Carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



O conjunto de dados é exibido, mostrando as primeiras 5 linhas para verificar se foi carregado corretamente. O arquivo contém várias colunas com características dos veículos, incluindo Consumo, Cilindros, Cilindradas, e HP.

#### Seleção de Colunas Relevantes
Para simplificar a análise, selecionamos apenas as colunas que serão utilizadas como variáveis independentes (Consumo, Cilindros, Cilindradas) e a variável dependente (HP).

In [0]:
#separa colunas
Carros = Carros_temp.select("Consumo","Cilindros","Cilindradas","HP")
Carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



As colunas selecionadas são exibidas, garantindo que estamos trabalhando apenas com as informações relevantes para o modelo de regressão.

#### Vetorização dos Atributos
Nesta etapa, utilizamos o VectorAssembler para transformar as colunas de características (Consumo, Cilindros, Cilindradas) em um único vetor chamado caracteristicas, que será utilizado como entrada para o modelo de regressão.

In [0]:
#vetoriza atributos
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")],outputCol="caracteristicas")
Carros = veccaracteristicas.transform(Carros)
Carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



O DataFrame agora inclui uma nova coluna chamada caracteristicas, que contém vetores numéricos com os valores de Consumo, Cilindros e Cilindradas para cada linha. Essa coluna será usada como features no treinamento do modelo.

#### Divisão dos Dados em Treino e Teste
Dividimos o conjunto de dados em duas partes: 70% para treinamento do modelo (CarrosTreino) e 30% para teste (CarrosTeste). Isso nos permitirá avaliar o desempenho do modelo em dados que ele não viu durante o treinamento.

In [0]:
#divide treino e teste
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3])
print(CarrosTreino.count())
print(CarrosTeste.count())

21
11


O número de registros em cada conjunto (treino e teste) é exibido, confirmando a divisão correta dos dados.

#### Treinamento do Modelo de Regressão Linear
Nesta etapa, utilizamos a classe LinearRegression para treinar o modelo de regressão linear com o conjunto de dados de treinamento. O modelo é ajustado para prever HP com base nas características vetorizadas.

In [0]:
#modelo
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(CarrosTreino)

#### Previsão com o Modelo Treinado
Após o treinamento, utilizamos o modelo para prever os valores de HP no conjunto de dados de teste. Essa etapa nos ajuda a verificar quão bem o modelo generaliza para novos dados.

In [0]:
#prever
previsao = modelo.transform(CarrosTeste)
previsao.show(11)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     26|        4|       1203| 91| [26.0,4.0,1203.0]|    90.14079421687|
|    133|        8|        350|245| [133.0,8.0,350.0]|204.69604951890366|
|    147|        8|        440|230| [147.0,8.0,440.0]|202.14955514242703|
|    164|        8|       2758|180|[164.0,8.0,2758.0]|188.25654575168463|
|    192|        8|        400|175| [192.0,8.0,400.0]|195.57463530234685|
|    197|        6|        145|175| [197.0,6.0,145.0]| 132.8279309262573|
|    214|        4|        121|109| [214.0,4.0,121.0]| 67.14613999731509|
|    228|        4|       1408| 95|[228.0,4.0,1408.0]| 58.74619379679286|
|    304|        4|        951|113| [304.0,4.0,951.0]| 49.54630770238324|
|    324|        4|        787| 66| [324.0,4.0,787.0]| 47.33916265514848|
|    339|        4|        711| 65| [3

As previsões de HP são exibidas ao lado dos valores reais de HP para os primeiros 11 registros do conjunto de teste. Isso nos permite comparar diretamente as previsões com os valores reais.

#### Avaliação da Performance do Modelo
Finalmente, avaliamos a performance do modelo utilizando a métrica de erro quadrático médio (RMSE). Um valor mais baixo de RMSE indica que o modelo tem uma precisão maior.

In [0]:
#avaliar performance
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse')
rmse = avaliar.evaluate(previsao)
print(rmse)

33.70121769541871


#### Conclusão
O valor de RMSE (Root Mean Square Error) encontrado foi de 33.70. Esse valor representa a média dos erros ao quadrado entre os valores preditos pelo modelo de regressão linear e os valores reais da variável HP no conjunto de dados de teste. Um RMSE menor indica que as previsões do modelo estão mais próximas dos valores reais, o que, em geral, reflete um modelo mais preciso.

No contexto dos dados automotivos que estamos analisando, um RMSE de 33.70 sugere que o modelo de regressão linear conseguiu capturar parte das relações entre as variáveis de entrada (Consumo, Cilindros, Cilindradas) e a potência do motor (HP), mas ainda há espaço para melhorias. Esse valor pode ser considerado relativamente elevado dependendo da escala da variável HP, o que indica que o modelo pode estar subestimando ou superestimando as previsões em uma margem significativa.

Para melhorar a performance do modelo, podemos explorar outras técnicas de regressão, como o uso de árvores de decisão ou florestas aleatórias, ajustar os parâmetros do modelo ou até mesmo incluir mais características que possam ajudar a explicar melhor a variável HP. Além disso, uma análise mais profunda dos dados, como a normalização das variáveis ou a remoção de outliers, também pode contribuir para a redução do erro.

Em notebooks futuros, podemos explorar essas alternativas e comparar os resultados para determinar a abordagem mais eficaz para prever a potência do motor dos veículos com base nas características disponíveis.